# M4L - Timbre Transfer 

"Have fun! And please feel free to hack this notebook to make your own creative interactions.“[- Magenta](https://colab.research.google.com/github/magenta/ddsp/blob/master/ddsp/colab/demos/timbre_transfer.ipynb)

This is such a hacked notebook. It contains a loop that searches a Google Drive folder for new files to process, and have a m4l-devices for settings and download of processed files.

### Instructions for running:

* Make sure to use a GPU runtime, click:  __Runtime >> Change Runtime Type >> GPU__
* Press ▶️ on the left of each of the cells
* View the code: Double-click any of the cells
* Hide the code: Double click the right side of the cell



Imports

In [0]:
#@title Install and Import

#@ markdown DInstall ddsp, define some helper functions, and download the model. This transfers a lot of data and _should take a minute or two_.
%tensorflow_version 2.x

print('Installing from pip package...')
!pip install -qU ddsp

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

# Ignore a bunch of deprecation warnings
import warnings
warnings.filterwarnings("ignore")

import copy
import os
import time 
import json
import crepe
import ddsp
import ddsp.training
from ddsp.colab import colab_utils
from ddsp.colab.colab_utils import (
    auto_tune, detect_notes, fit_quantile_transform, 
    get_tuning_factor, download, play, record, 
    specplot, upload, DEFAULT_SAMPLE_RATE)
import gin
import gin
from google.colab import files
import librosa
import matplotlib.pyplot as plt
import numpy as np
import tensorflow.compat.v2 as tf
import tensorflow_datasets as tfds

# Helper Functions
sample_rate = DEFAULT_SAMPLE_RATE  # 16000

import base64
import io
import tempfile

import ddsp
from IPython import display
import librosa
import matplotlib.pyplot as plt
import numpy as np
import pickle
from pydub import AudioSegment
from scipy.io import wavfile
import tensorflow.compat.v2 as tf

from google.colab import files
from google.colab import output
download = files.download

DEFAULT_SAMPLE_RATE = 16000
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import json, time
print('Done!')




##Login for authentication for Google Drive access

In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


# Load functionality from DDSP 'Timbre Transfer Demo Notebook'

Essentially copy/paste from the DDSP 'Timbre Transfer Demo Notebook', made to work for this. Added functions for file management. 

In [0]:
#@title Whole lotta code
gdID = None
file_list = drive.ListFile({'q': "'root' in parents and trashed=false and title = 'M4L-Timbre-Transfer-Folder'"}).GetList()
for file1 in file_list:
  # print("FOLDER", file1)
  print('title: %s, id: %s' % (file1['title'], file1['id']))
  gdID = file1['id']


audio = None
audio_features = None
audio_features_mod = None
def p4load(fileName):
  global audio, audio_features, audio_features_mod
  anp, sr = load_audio(fileName)
  print("anp", anp)
  audio = anp[np.newaxis, :]
  specplot(audio)
  play(audio)

  # Setup the session.
  ddsp.spectral_ops.reset_crepe()

  start_time = time.time()
  audio_features = ddsp.training.metrics.compute_audio_features(audio)
  audio_features['loudness_db'] = audio_features['loudness_db'].astype(np.float32)
  audio_features_mod = None
  print('Audio features took %.1f seconds' % (time.time() - start_time))


TRIM = -15


#modAudio = 0;
def load_audio(path):
   ##3 """Load audio into numpy array, cache it so on reload script music doesn't need to reload"""
    # TODO : should detect format to pass sample rate when enabling mp3, 16000 for mp3
    audio_np, unused_sr = librosa.core.load(path, sr=16000)
    return audio_np.astype(np.float32), unused_sr

# @title P4 MODEL
    
# model = null
# 'Violin' #@param ['Violin', 'Flute', 'Flute2', 'Trumpet', 'Tenor_Saxophone','Upload your own (checkpoint folder as .zip)']
# MODEL = model

def p4model(m):
    global audio_features_mod, audio, audio_features
    global model, MODEL
    model = m
    MODEL = m
    #audio_features = modified
    GCS_CKPT_DIR = 'gs://ddsp/models/tf2'
    print("m", m, "model", model, "MODEL", MODEL)
    if model in ('Violin', 'Flute', 'Flute2', 'Trumpet', 'Tenor_Saxophone'):
        # Pretrained models.
        PRETRAINED_DIR = '/content/pretrained'
        # Copy over from gs:// for faster loading.
        !rm -r $PRETRAINED_DIR &> /dev/null
        !mkdir $PRETRAINED_DIR &> /dev/null
        GCS_CKPT_DIR = 'gs://ddsp/models/tf2'
        model_dir = os.path.join(GCS_CKPT_DIR, 'solo_%s_ckpt' % model.lower())
        
        !gsutil cp $model_dir/* $PRETRAINED_DIR &> /dev/null
        model_dir = PRETRAINED_DIR
        gin_file = os.path.join(model_dir, 'operative_config-0.gin')

    # Load the dataset statistics.
    DATASET_STATS = None
    dataset_stats_file = os.path.join(model_dir, 'dataset_statistics.pkl')
    print(f'Loading dataset statistics from {dataset_stats_file}')
    try:
      if tf.io.gfile.exists(dataset_stats_file):
        with tf.io.gfile.GFile(dataset_stats_file, 'rb') as f:
          DATASET_STATS = pickle.load(f)
    except Exception as err:
      print('Loading dataset statistics from pickle failed: {}.'.format(err))

      
    # Parse gin config,
    with gin.unlock_config():
      gin.parse_config_file(gin_file, skip_unknown=True)
    
    # Assumes only one checkpoint in the folder, 'ckpt-[iter]`.
    ckpt_files = [f for f in tf.io.gfile.listdir(model_dir) if 'ckpt' in f]
    ckpt_name = ckpt_files[0].split('.')[0]
    ckpt = os.path.join(model_dir, ckpt_name)
    
    # Ensure dimensions and sampling rates are equal
    time_steps_train = gin.query_parameter('DefaultPreprocessor.time_steps')
    n_samples_train = gin.query_parameter('Additive.n_samples')
    hop_size = int(n_samples_train / time_steps_train)
    
    time_steps = int(audio.shape[1] / hop_size)
    n_samples = time_steps * hop_size
    
    # print("===Trained model===")
    # print("Time Steps", time_steps_train)
    # print("Samples", n_samples_train)
    # print("Hop Size", hop_size)
    # print("\n===Resynthesis===")
    # print("Time Steps", time_steps)
    # print("Samples", n_samples)
    # print('')
    
    gin_params = [
        'RnnFcDecoder.input_keys = ("f0_scaled", "ld_scaled")',
        'Additive.n_samples = {}'.format(n_samples),
        'FilteredNoise.n_samples = {}'.format(n_samples),
        'DefaultPreprocessor.time_steps = {}'.format(time_steps),
    ]
    
    with gin.unlock_config():
      gin.parse_config(gin_params)
    
    
    # Trim all input vectors to correct lengths 
    for key in ['f0_hz', 'f0_confidence', 'loudness_db']:
      audio_features[key] = audio_features[key][:time_steps]
    audio_features['audio'] = audio_features['audio'][:, :n_samples]
    
    
    # Set up the model just to predict audio given new conditioning
    model = ddsp.training.models.Autoencoder()
    model.restore(ckpt)
    
    # Build model by running a batch through it.
    start_time = time.time()
    _ = model(audio_features, training=False)
    print('Restoring model took %.1f seconds' % (time.time() - start_time))
    #return audio_features
print('DONE')# @title P4 MODEL
    
# model = null
# 'Violin' #@param ['Violin', 'Flute', 'Flute2', 'Trumpet', 'Tenor_Saxophone','Upload your own (checkpoint folder as .zip)']
# MODEL = model
DATASET_STATS = None
def p4model(m):
    global audio_features_mod, audio, audio_features, DATASET_STATS
    global model, MODEL
    model = m
    MODEL = m
    #audio_features = modified
    GCS_CKPT_DIR = 'gs://ddsp/models/tf2'
    print("m", m, "model", model, "MODEL", MODEL)
    if model in ('Violin', 'Flute', 'Flute2', 'Trumpet', 'Tenor_Saxophone'):
        # Pretrained models.
        PRETRAINED_DIR = '/content/pretrained'
        # Copy over from gs:// for faster loading.
        !rm -r $PRETRAINED_DIR &> /dev/null
        !mkdir $PRETRAINED_DIR &> /dev/null
        GCS_CKPT_DIR = 'gs://ddsp/models/tf2'
        model_dir = os.path.join(GCS_CKPT_DIR, 'solo_%s_ckpt' % model.lower())
        
        !gsutil cp $model_dir/* $PRETRAINED_DIR &> /dev/null
        model_dir = PRETRAINED_DIR
        gin_file = os.path.join(model_dir, 'operative_config-0.gin')
    
        # Load the dataset statistics.
    DATASET_STATS = None
    dataset_stats_file = os.path.join(model_dir, 'dataset_statistics.pkl')
    print(f'Loading dataset statistics from {dataset_stats_file}')
    try:
      if tf.io.gfile.exists(dataset_stats_file):
        with tf.io.gfile.GFile(dataset_stats_file, 'rb') as f:
          DATASET_STATS = pickle.load(f)
    except Exception as err:
      print('Loading dataset statistics from pickle failed: {}.'.format(err))

    # Parse gin config,
    with gin.unlock_config():
      gin.parse_config_file(gin_file, skip_unknown=True)
    
    # Assumes only one checkpoint in the folder, 'ckpt-[iter]`.
    ckpt_files = [f for f in tf.io.gfile.listdir(model_dir) if 'ckpt' in f]
    ckpt_name = ckpt_files[0].split('.')[0]
    ckpt = os.path.join(model_dir, ckpt_name)
    
    # Ensure dimensions and sampling rates are equal
    time_steps_train = gin.query_parameter('DefaultPreprocessor.time_steps')
    n_samples_train = gin.query_parameter('Additive.n_samples')
    hop_size = int(n_samples_train / time_steps_train)
    
    time_steps = int(audio.shape[1] / hop_size)
    n_samples = time_steps * hop_size
    
    # print("===Trained model===")
    # print("Time Steps", time_steps_train)
    # print("Samples", n_samples_train)
    # print("Hop Size", hop_size)
    # print("\n===Resynthesis===")
    # print("Time Steps", time_steps)
    # print("Samples", n_samples)
    # print('')
    
    gin_params = [
        'RnnFcDecoder.input_keys = ("f0_scaled", "ld_scaled")',
        'Additive.n_samples = {}'.format(n_samples),
        'FilteredNoise.n_samples = {}'.format(n_samples),
        'DefaultPreprocessor.time_steps = {}'.format(time_steps),
    ]
    
    with gin.unlock_config():
      gin.parse_config(gin_params)
    
    
    # Trim all input vectors to correct lengths 
    for key in ['f0_hz', 'f0_confidence', 'loudness_db']:
      audio_features[key] = audio_features[key][:time_steps]
    audio_features['audio'] = audio_features['audio'][:, :n_samples]
    
    
    # Set up the model just to predict audio given new conditioning
    model = ddsp.training.models.Autoencoder()
    model.restore(ckpt)
    
    # Build model by running a batch through it.
    start_time = time.time()
    _ = model(audio_features, training=False)
    print('Restoring model took %.1f seconds' % (time.time() - start_time))
    #return audio_features
#@ title p4 MODIFY { output-height: 400 }
## Helper functions.
def shift_ld(audio_features, ld_shift=0.0):
  """Shift loudness by a number of ocatves."""
  audio_features['loudness_db'] += ld_shift
  return audio_features


def shift_f0(audio_features, f0_octave_shift=0.0):
  """Shift f0 by a number of ocatves."""
  audio_features['f0_hz'] *= 2.0 ** (f0_octave_shift)
  audio_features['f0_hz'] = np.clip(audio_features['f0_hz'], 
                                    0.0, 
                                    librosa.midi_to_hz(110.0))
  return audio_features






def p4modify( octave, loudness, threshold, autoadjust,  autotune, quiet):
  global audio_features_mod, audio_features, DATASET_STATS
  audio_features_mod = {k: v.copy() for k, v in audio_features.items()}
  audio_features_mod = shift_ld(audio_features_mod, loudness)
  audio_features_mod = shift_f0(audio_features_mod, octave)
  #return audio_features_mod
  if autoadjust and DATASET_STATS is not None:
    # Detect sections that are "on".
    mask_on, note_on_value = detect_notes(audio_features['loudness_db'],
                                          audio_features['f0_confidence'],
                                          threshold)

    if np.any(mask_on):
      # Shift the pitch register.
      target_mean_pitch = DATASET_STATS['mean_pitch']
      pitch = ddsp.core.hz_to_midi(audio_features['f0_hz'])
      mean_pitch = np.mean(pitch[mask_on])
      p_diff = target_mean_pitch - mean_pitch
      p_diff_octave = p_diff / 12.0
      round_fn = np.floor if p_diff_octave > 1.5 else np.ceil
      p_diff_octave = round_fn(p_diff_octave)
      audio_features_mod = shift_f0(audio_features_mod, p_diff_octave)


      # Quantile shift the note_on parts.
      _, loudness_norm = colab_utils.fit_quantile_transform(
          audio_features['loudness_db'],
          mask_on,
          inv_quantile=DATASET_STATS['quantile_transform'])

      # Turn down the note_off parts.
      mask_off = np.logical_not(mask_on)
      loudness_norm[mask_off] -=  quiet * (1.0 - note_on_value[mask_off][:, np.newaxis])
      loudness_norm = np.reshape(loudness_norm, audio_features['loudness_db'].shape)
      
      audio_features_mod['loudness_db'] = loudness_norm 

      # Auto-tune.
      if autotune:
        f0_midi = np.array(ddsp.core.hz_to_midi(audio_features_mod['f0_hz']))
        tuning_factor = get_tuning_factor(f0_midi, audio_features_mod['f0_confidence'], mask_on)
        f0_midi_at = auto_tune(f0_midi, tuning_factor, mask_on, amount=autotune)
        audio_features_mod['f0_hz'] = ddsp.core.midi_to_hz(f0_midi_at)
    

  
print('DONE')


#^^^^MODIFIED NOTEBOOK

def create_audio_file(array_of_floats, path):
    if len(array_of_floats.shape) == 2:
        array_of_floats = array_of_floats[0]
    normalizer = float(np.iinfo(np.int16).max)
    array_of_ints = np.array(
        np.asarray(array_of_floats) * normalizer, dtype=np.int16)
    memfile = io.BytesIO()
    wavfile.write(path, sample_rate, array_of_ints)
    memfile.close()
    print("SAVED FILE")


def p4makeSave2(path):
  global audio_features_mod, audio_features, audio, count, gdID
  af = audio_features if audio_features_mod is None else audio_features_mod
  # Run a batch of predictions.
  start_time = time.time()
  # print(mAudio, type(mAudio))
  print(af)
  audio_gen = model(af, training=False)
  play(audio_gen)
  specplot(audio_gen)
  print('Prediction took %.1f seconds' % (time.time() - start_time))
  create_audio_file(audio_gen, path)
  # uploaded = drive.CreateFile({'title': 'Sample upload.txt'})
  # uploaded.SetContentString('Sample upload file content')
  # uploaded.Upload()
  # print('Uploaded file with ID {}'.format(uploaded.get('id')))
  TTdfile = drive.CreateFile(
            {'parents': [{'id': gdID}], 'title': 'ddspd' + str(count) + '.wav'})
            
  TTdfile.SetContentFile(path)
  TTdfile.Upload()
print('DONE')

This continously checks for new files in Google Drive, processes them, and uploaded the transferred audio.

In [0]:
#@title LOOP
count = 0
found = False
while True:
  global count, found
  # FIND JSON
  if not found:
    file_list = drive.ListFile(
              {'q':  "'" + gdID + "'" + " in parents and trashed=false and title = '" + "settings" + str(count) + ".json" + "'"}).GetList()
    for file1 in file_list:
        print('title: %s, id: %s' % (file1['title'], file1['id']))
        # dlfile = drive.CreateFile({'id': file1['id']})
        # dlfile.GetContentFile(file1)
        downloaded = drive.CreateFile({'id': file1['id']})
        print('Downloaded content "{}"'.format(downloaded.GetContentString()))
        jf = json.loads(downloaded.GetContentString())
        print('time passed:', ((time.time()) % 86400) - jf["time"])
        found = True

  # LOAD AUDIO
  file_list = drive.ListFile(
            {'q':  "'" + gdID + "'" + " in parents and trashed=false and title = '" + "sendAudio" + str(count) + ".wav" + "'"}).GetList()
  for file1 in file_list:
      print('title: %s, id: %s' % (file1['title'], file1['id']))
      # dlfile = drive.CreateFile({'id': file1['id']})
      # dlfile.GetContentFile(file1)
      aud = drive.CreateFile({'id': file1['id']})
      aud.GetContentFile("ddsp" + str(count) + ".wav")
      
      
      p4load("ddsp" + str(count) + ".wav")
      play(audio)
      print("model from json is", jf["model"])  
      p4model(jf["model"])
      
      p4modify( jf["octave"], jf["loudness"], jf["threshold"], jf["auto"], jf["autotune"],jf["quiet"] )
      
      p4makeSave2(("audio-transferred" + str(count) + '.wav'))
      count = count + 1
      found = False
      